In [1]:
from flask import Flask, request, flash, redirect, url_for, render_template, send_file

import numpy as np
import pandas as pd
import sys
import os
import json
import requests
import urllib.request

UPLOAD_FOLDER = './uploads'
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)
ALLOWED_EXTENSIONS = {'txt', 'pdf', 'png', 'jpg', 'jpeg'}

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

In [ ]:
# ------------------------------------- RUN APP -------------------------------------------------
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

@app.route('/', methods=['GET'])
def upload_file():
    return render_template('initial.html')

@app.route('/', methods=['POST'])
def main_page():
    if 'file' not in request.files:
        flash('No file part')
        return redirect(request.url)
    file = request.files['file']
    # If the user does not select a file, the browser submits an
    # empty file without a filename.
    if file.filename == '':
        flash('No selected file')
        return redirect(request.url)
    if file:
        if allowed_file(file.filename):
            # если все ок, то сохраняем файл по директории UPLOAD_FOLDER
            filename = file.filename
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            # тут делаем бэкэнд ? 
            # <backend code here>
            return redirect('/processed/')
        else:
            flash('This file is not allowed')
            return redirect(request.url)
    return redirect(request.url)

@app.route('/processed/')
def processed():
    try:
        return send_file(os.path.join(app.config['UPLOAD_FOLDER'], 'exit.zip'))
    except Exception as e:
        return str(e)


if __name__=='__main__': 
    #app.run(host='0.0.0.0', port='9889', debug=True)
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Oct/2021 22:53:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2021 22:53:44] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [20/Oct/2021 22:53:44] "GET /processed/ HTTP/1.1" 200 -
